In [1]:
# load libraries
import lstm
import torch
import queue
import threading
import numpy as np 
import pandas as pd
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
x_train = pd.read_csv('../../../data/pecan/dataset4/x_train_05_27_24.csv')
y_train = pd.read_csv('../../../data/pecan/dataset4/y_train_05_27_24.csv')
x_test = pd.read_csv('../../../data/pecan/dataset4/x_test_05_27_24.csv')
y_test = pd.read_csv('../../../data/pecan/dataset4/y_test_05_27_24.csv')

In [3]:
X = torch.tensor(x_train.values, dtype=torch.float32)
Y = torch.tensor(y_train.values, dtype=torch.float32)

sequence_length = 1  # xrbitrary number chosen


def create_sequences(X, Y, seq_length):
    xs, ys = [], []
    for i in range(len(X) - seq_length):
        x = X[i:i+seq_length]
        y = Y[i+seq_length]
        xs.append(x)
        ys.append(y)
    return torch.stack(xs), torch.stack(ys)

X, y = create_sequences(X, Y, sequence_length)


print(X.shape, y.shape) 


torch.Size([15, 1, 122976]) torch.Size([15, 1])


In [4]:
dataset = TensorDataset(X, y)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
new_val_size = int(0.8 * val_size)
test_size = val_size - new_val_size

generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)
val_dataset, test_dataset = random_split(val_dataset, [new_val_size, test_size], generator=generator)

print(f'Train Dataset size: {len(train_dataset)} \nValidation Dataset size {len(val_dataset)}\nTest Dataset size {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Train Dataset size: 12 
Validation Dataset size 2
Test Dataset size 1


In [5]:
# load libraries
import json
import torch
import queue
import threading
import numpy as np 
import pandas as pd
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split




class LSTM(nn.Module):
    def __init__(self, input_size=122976, hidden_size=1280, num_layers=20, patience=10, min_delta=10):
        super(LSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
        self.activations = []
        #member vars for the early stopper
        self.patience = patience
        self.min_delta = min_delta
        self.min_validation_loss = float('inf')

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        self.activations.append(out.squeeze())
        return out.squeeze()

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss - self.min_delta):
            self.counter +=1
            if self.counter >= self.patience:
                return True
        return False

class Trainer():
    def __init__(self, model, train_loader, validation_loader, device, criterion=nn.MSELoss(), lr=0.001, num_epochs=10000):
        self.model = model
        self.train_loader = train_loader
        self.validation_loader = validation_loader
        self.criterion = criterion
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = device
        self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr)
        self.train_losses = []
        self.val_losses = []
        self.loss = None

    def train_one_epoch(self):
        self.model.train()
        epoch_loss = 0
        for data, targets in self.train_loader:
            data = data.to(self.device)
            targets = targets.to(self.device)

            self.optimizer.zero_grad()
            outputs = self.model(data)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / len(self.train_loader)

    def val_one_epoch(self):
        self.model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for data, targets in self.validation_loader:
                data = data.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(data)
                loss = self.criterion(outputs, targets)
                epoch_loss += loss.item()
        return epoch_loss / len(self.validation_loader)
        
    def train(self):
        for epoch in np.arange(self.num_epochs):
            train_loss = self.train_one_epoch()
            val_loss = self.val_one_epoch()
            
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)

            if(epoch % 100 == 0):
                print(f'Epoch: {epoch} Train Loss: {train_loss} Validation Loss: {val_loss}')

            if self.model.early_stop(val_loss):
                break
            
        print('#'*100)
        print(f'Final Epoch: {epoch} Train Loss: {train_loss} Validation Loss: {val_loss}')
        return self.model, self.train_losses, self.val_losses


In [6]:
# set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device set to:", device)

model = LSTM().to(device)
# trainer = Trainer(model, train_loader, val_loader, device)
# model, train_losses, val_losses = trainer.train()

Device set to: cuda


In [7]:
import torchinfo

torchinfo.summary(model, input_size=(1,32,122976))

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─LSTM: 1-1                              [1, 32, 1280]             885,432,320
├─Linear: 1-2                            [1, 1]                    1,281
Total params: 885,433,601
Trainable params: 885,433,601
Non-trainable params: 0
Total mult-adds (G): 28.33
Input size (MB): 15.74
Forward/backward pass size (MB): 0.33
Params size (MB): 3541.73
Estimated Total Size (MB): 3557.80